# cloudflare
> Cloudflare DNS and tunnel management via the official Python SDK

In [ ]:
#| default_exp cloudflare

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import os, base64

## CF

Thin wrapper around the official `cloudflare` Python SDK. Token is read from the `CLOUDFLARE_API_TOKEN` environment variable by default.

In [ ]:
#| export
class CF:
    'Cloudflare API wrapper using the official cloudflare Python SDK'
    def __init__(self, token=None):
        from cloudflare import Cloudflare
        self._c = Cloudflare(api_token=token or os.environ['CLOUDFLARE_API_TOKEN'])

    def zones(self) -> list:
        'List all zones'
        return [z.model_dump() for z in self._c.zones.list()]

    def zone_id(self, domain) -> str:
        'Get zone_id for domain'
        for z in self._c.zones.list(name=domain):
            return z.id
        raise ValueError(f'Zone not found: {domain}')

    def dns_records(self, zone_id) -> list:
        'List DNS records for a zone'
        return [r.model_dump() for r in self._c.dns.records.list(zone_id=zone_id)]

    def create_record(self, zone_id, type, name, content, ttl=1, proxied=False) -> dict:
        'Create a DNS record'
        r = self._c.dns.records.create(
            zone_id=zone_id, type=type, name=name, content=content, ttl=ttl, proxied=proxied)
        return r.model_dump()

    def delete_record(self, zone_id, record_id) -> None:
        'Delete a DNS record'
        self._c.dns.records.delete(record_id, zone_id=zone_id)

    def upsert_record(self, domain, name, content, type='A', proxied=False) -> dict:
        'Create or replace DNS record. Deletes existing same-name+type record first.'
        zid = self.zone_id(domain)
        full = name if '.' in name else f'{name}.{domain}'
        for r in self._c.dns.records.list(zone_id=zid):
            if r.name == full and r.type == type:
                self.delete_record(zid, r.id)
        return self.create_record(zid, type, name, content, proxied=proxied)

    def account_id(self) -> str:
        'Get first account ID'
        for a in self._c.accounts.list():
            return a.id
        raise ValueError('No accounts found')

    def tunnels(self, account_id=None) -> list:
        'List Cloudflare tunnels'
        aid = account_id or self.account_id()
        return [t.model_dump() for t in self._c.zero_trust.tunnels.cloudflared.list(account_id=aid)]

    def create_tunnel(self, name, account_id=None) -> dict:
        'Create a new Cloudflare tunnel'
        aid = account_id or self.account_id()
        secret = base64.urlsafe_b64encode(os.urandom(32)).decode()
        t = self._c.zero_trust.tunnels.cloudflared.create(
            account_id=aid, name=name, tunnel_secret=secret)
        return t.model_dump()

    def delete_tunnel(self, tunnel_id, account_id=None) -> None:
        'Delete a Cloudflare tunnel'
        aid = account_id or self.account_id()
        self._c.zero_trust.tunnels.cloudflared.delete(tunnel_id, account_id=aid)

    def tunnel_token(self, tunnel_id, account_id=None) -> str:
        'Get tunnel token string for use with cloudflared_svc() / CF_TUNNEL_TOKEN env var'
        aid = account_id or self.account_id()
        result = self._c.zero_trust.tunnels.cloudflared.token.get(tunnel_id, account_id=aid)
        return result if isinstance(result, str) else result.token

In [ ]:
# Instantiation test: object created without raising when given a mock token
import os as _os

cf = CF(token='fake-token-for-testing')
assert cf._c is not None
print('CF instantiation OK')

# Real API calls gated on CF_TEST=1
if _os.environ.get('CF_TEST') == '1':
    cf = CF()  # uses CLOUDFLARE_API_TOKEN from env
    zs = cf.zones()
    names = [z['name'] for z in zs]
    print(f'zones: {names}')
    print('CF.zones() OK')
else:
    print('Skipping real API tests (set CF_TEST=1 to enable)')

In [ ]:
#| export
def dns_record(domain, name, content, type='A', proxied=False, token=None) -> dict:
    'Upsert a DNS record. Reads CLOUDFLARE_API_TOKEN from env if token not given.'
    return CF(token=token).upsert_record(domain, name, content, type=type, proxied=proxied)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()